In [ ]:
import requests
import pandas as pd

# Configuration
INPUT_FILE = "https://raw.githubusercontent.com/Jeffateth/AllergenPredict/main/allergen_data_with_full_sequences.csv"  # change input file
SEQUENCE_COLUMN = (
    "full_parent_protein_sequence"  # specify in which column the sequence is saved
)


def search_rcsb_by_sequence(sequence):
    """Search the RCSB PDB by an amino acid sequence using a POST request."""
    url = "https://search.rcsb.org/rcsbsearch/v2/query"

    query = {
        "query": {
            "type": "terminal",
            "service": "sequence",
            "parameters": {
                "evalue_cutoff": 0.1,
                "identity_cutoff": 0.9,
                "target": "pdb_protein_sequence",
                "value": sequence,
            },
        },
        "return_type": "entry",
    }

    headers = {"Content-Type": "application/json"}

    response = requests.post(url, json=query, headers=headers)

    if response.status_code == 200:
        data = response.json()
        pdb_ids = [result["identifier"] for result in data.get("result_set", [])]
        return pdb_ids
    elif response.status_code == 204:
        return []  # No content (no matches found)
    else:
        print(f"Error {response.status_code} for sequence: {sequence}")
        return []


def main():
    # Read the input CSV
    df = pd.read_csv(INPUT_FILE)

    # Check if the expected column exists
    if SEQUENCE_COLUMN not in df.columns:
        print(f"Column '{SEQUENCE_COLUMN}' not found in the input file.")
        return

    # Work with the first 1000 sequences and filter out short ones
    # df_subset = df.head(1000).copy()
    df_subset = df[df[SEQUENCE_COLUMN].str.len() >= 20]

    # Create a new column to hold matching PDB codes
    df_subset["pdb_matches"] = df_subset[SEQUENCE_COLUMN].apply(
        lambda seq: search_rcsb_by_sequence(seq)
    )

    # Save results
    df_subset.to_csv("sequence_pdb_matches.csv", index=False)
    print("Results saved to sequence_pdb_matches.csv")


if __name__ == "__main__":
    main()

Results saved to sequence_pdb_matches.csv


In [ ]:
df_matches = pd.read_csv("sequence_pdb_matches.csv")
print(df_matches)

                    epitope_sequence  \
0    FGGRAEWGTNTADNDDTDGNGHGTHTASTAA   
1               TEEEKNRLNFLKKISQRYQK   
2      TAIFQDTVRAEMTKVLAPAFKKELERNNQ   
3               RQRVEQEQEQEQDEYPYSQR   
4           PKHADADNILVIQQGQATVTVANG   
..                               ...   
155             TALKKAITAMSQAQKAAKPA   
156             ELFRQFYQLDAYPSGAWYYV   
157             KAKFETFKKEMKAKEAELAK   
158         ARQQWELQEDRRCQSQLERANLRP   
159             PYSPSQDPDRRDPYSPSPYD   

                                   protein_url  label  \
0        http://www.uniprot.org/uniprot/P9WEW4      0   
1        http://www.uniprot.org/uniprot/P02663      0   
2        http://www.uniprot.org/uniprot/P49273      1   
3        http://www.uniprot.org/uniprot/Q9SQH1      1   
4        http://www.uniprot.org/uniprot/B3IXL2      1   
..                                         ...    ...   
155      http://www.uniprot.org/uniprot/P22286      0   
156      http://www.uniprot.org/uniprot/P02662      1   
157  h

In [11]:
!ls

sample_data


In [13]:
#!/usr/bin/env python3
"""
Comprehensive per‑residue structural descriptor calculation from a PDB,
with automatic installation of missing dependencies, and Biopython ≥ 1.80 support.

Usage:
    python analyze_protein.py <path/to/structure.pdb>
"""

import sys
import subprocess

# --------------------------------------------------
# 1) Auto‑install missing packages
# --------------------------------------------------


def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])


for pkg in ("numpy", "pandas", "mdtraj", "biopython"):
    try:
        __import__(pkg)
    except ImportError:
        print(f"Package '{pkg}' not found. Installing...")
        install(pkg)

# Now safe to import everything
import mdtraj as md
import numpy as np
import pandas as pd
from Bio.PDB import PDBParser, HSExposureCA
from Bio.Data.PDBData import protein_letters_3to1

# --------------------------------------------------
# 2) Constants
# --------------------------------------------------
MAX_ASA = {
    "A": 121,
    "R": 265,
    "N": 187,
    "D": 187,
    "C": 148,
    "Q": 214,
    "E": 223,
    "G": 97,
    "H": 216,
    "I": 195,
    "L": 191,
    "K": 230,
    "M": 203,
    "F": 228,
    "P": 154,
    "S": 143,
    "T": 163,
    "W": 264,
    "Y": 255,
    "V": 165,
}

HYDRO = {
    "A": 1.8,
    "R": -4.5,
    "N": -3.5,
    "D": -3.5,
    "C": 2.5,
    "Q": -3.5,
    "E": -3.5,
    "G": -0.4,
    "H": -3.2,
    "I": 4.5,
    "L": 3.8,
    "K": -3.9,
    "M": 1.9,
    "F": 2.8,
    "P": -1.6,
    "S": -0.8,
    "T": -0.7,
    "W": -0.9,
    "Y": -1.3,
    "V": 4.2,
}

VDW_RADII = {"H": 1.2, "C": 1.7, "N": 1.55, "O": 1.52, "S": 1.8}

# --------------------------------------------------
# 3) Helper to map 3‑letter → 1‑letter
# --------------------------------------------------


def resname_to_one(resname):
    """Convert a three‑letter residue name to one‑letter code, default 'X'."""
    return protein_letters_3to1.get(resname.upper(), "X")


# --------------------------------------------------
# 4) Core functions
# --------------------------------------------------


def load_structures(pdb_path):
    traj = md.load(pdb_path)
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure("X", pdb_path)
    return traj, structure


def compute_sasa(traj, structure):
    sasa_nm2 = md.shrake_rupley(traj, probe_radius=0.14, mode="residue")
    sasa = sasa_nm2[0] * 100.0  # nm² → Å²
    norm = []
    for res in structure.get_residues():
        aa1 = resname_to_one(res.get_resname())
        max_ = MAX_ASA.get(aa1, np.nan)
        norm.append(sasa[res.id[1] - 1] / max_ if max_ else np.nan)
    return sasa, np.array(norm)


def compute_torsions(traj, structure):
    phi_idx, phi_vals = md.compute_phi(traj)
    psi_idx, psi_vals = md.compute_psi(traj)
    om_idx, om_vals = md.compute_omega(traj)
    chi1_idx, chi1_vals = md.compute_chi1(traj)

    phi = dict(zip(phi_idx[:, 1], phi_vals[0]))
    psi = dict(zip(psi_idx[:, 1], psi_vals[0]))
    omega = dict(zip(om_idx[:, 1], om_vals[0]))
    chi1 = dict(zip(chi1_idx[:, 0], chi1_vals[0]))

    phis = []
    psis = []
    omegas = []
    chis = []
    for res in structure.get_residues():
        i = res.id[1] - 1
        phis.append(phi.get(i, np.nan))
        psis.append(psi.get(i, np.nan))
        omegas.append(omega.get(i, np.nan))
        chis.append(chi1.get(i, np.nan))
    return (np.array(phis), np.array(psis), np.array(omegas), np.array(chis))


def compute_hbonds(traj, structure):
    hbonds = md.baker_hubbard(traj, periodic=False)[0]
    counts = {r.id[1]: 0 for r in structure.get_residues()}
    for d, a, *_ in hbonds:
        counts[traj.topology.atom(d).residue.id] += 1
        counts[traj.topology.atom(a).residue.id] += 1
    return np.array([counts[r.id[1]] for r in structure.get_residues()])


def compute_salt_bridges(traj, structure, cutoff=0.4):
    neg = traj.topology.select("resname ASP GLU and name OD1 OD2 OE1 OE2")
    pos = traj.topology.select("resname LYS ARG HIS and name NZ NH1 NH2 NE")
    d = md.compute_distances(traj, np.array([[i, j] for i in neg for j in pos]))[0]
    pairs = np.where(d < cutoff)
    counts = {r.id[1]: 0 for r in structure.get_residues()}
    for ni, pi in zip(*pairs):
        ri = traj.topology.atom(neg[ni]).residue.id
        rj = traj.topology.atom(pos[pi]).residue.id
        counts[ri] += 1
        counts[rj] += 1
    return np.array([counts[r.id[1]] for r in structure.get_residues()])


def compute_ss_bridges(traj, structure, cutoff=0.22):
    sg = traj.topology.select("resname CYS and name SG")
    d = md.compute_distances(traj, np.array([[i, j] for i in sg for j in sg]))[0]
    pairs = np.where((d < cutoff) & (d > 0))
    bonded = set()
    for i, j in zip(*pairs):
        bonded.update(
            [traj.topology.atom(sg[i]).residue.id, traj.topology.atom(sg[j]).residue.id]
        )
    return np.array([1 if r.id[1] in bonded else 0 for r in structure.get_residues()])


def compute_pi_pi(traj, structure, cutoff=0.7):
    arom = {
        "PHE": ("CG", "CD1", "CD2", "CE1", "CE2", "CZ"),
        "TYR": ("CG", "CD1", "CD2", "CE1", "CE2", "CZ"),
        "TRP": ("CG", "CD1", "CD2", "NE1", "CE2", "CE3", "CZ2", "CZ3", "CH2"),
    }
    cents = {}
    for chain in structure:
        for res in chain:
            name = res.get_resname()
            if name in arom:
                coords = np.array(
                    [a.get_vector() for a in res if a.get_name() in arom[name]]
                )
                cents[res.id[1]] = coords.mean(axis=0)
    counts = {r.id[1]: 0 for r in structure.get_residues()}
    keys = list(cents)
    for i in range(len(keys)):
        for j in range(i + 1, len(keys)):
            if np.linalg.norm(cents[keys[i]] - cents[keys[j]]) < cutoff * 10:
                counts[keys[i]] += 1
                counts[keys[j]] += 1
    return np.array([counts[r.id[1]] for r in structure.get_residues()])


def compute_vdw_contacts(traj, structure, extra=0.05):
    coords = traj.xyz[0] * 10
    elems = [a.element.symbol for a in traj.topology.atoms]
    radii = np.array([VDW_RADII.get(e, 1.5) for e in elems])
    pairs = np.array(
        [[i, j] for i in range(len(coords)) for j in range(i + 1, len(coords))]
    )
    dists = np.linalg.norm(coords[pairs[:, 0]] - coords[pairs[:, 1]], axis=1)
    thresh = radii[pairs[:, 0]] + radii[pairs[:, 1]] + extra
    hits = pairs[dists < thresh]
    resmap = {a.index: a.residue.id[1] for a in traj.topology.atoms}
    conts = {r.id[1]: set() for r in structure.get_residues()}
    for i, j in hits:
        ri, rj = resmap[i], resmap[j]
        if ri != rj:
            conts[ri].add(rj)
            conts[rj].add(ri)
    return np.array([len(conts[r.id[1]]) for r in structure.get_residues()])


def compute_packing_density(traj, structure, r=1.0):
    ca = traj.topology.select("name CA")
    coords = traj.xyz[0]
    vol = 4 / 3 * np.pi * r**3 * 1e3
    pd = []
    for idx, res in enumerate(structure.get_residues()):
        d = np.linalg.norm((coords - coords[0][ca[idx]]), axis=2) * 10
        pd.append((np.sum(d < r * 10) - 1) / vol)
    return np.array(pd)


def compute_depth_protrusion(structure):
    atoms = np.array([a.get_vector() for a in structure.get_atoms()])
    com = atoms.mean(axis=0)
    depths, protr = [], []
    for res in structure.get_residues():
        coords = np.array([a.get_vector() for a in res if a.element.symbol != "H"])
        d = np.linalg.norm(coords - com, axis=1)
        depths.append(d.mean())
        protr.append(d.max() - d.mean())
    return np.array(depths), np.array(protr)


def compute_half_sphere_exposure(structure):
    model = structure[0]
    hs = HSExposureCA(model)
    up, down = [], []
    for res in model:
        u, d = hs[(res, "CA")]
        up.append(u)
        down.append(d)
    return np.array(up), np.array(down)


def compute_contact_map(traj, thr=0.8):
    ca = traj.topology.select("name CA")
    pairs = np.array([[i, j] for i in ca for j in ca if i < j])
    d = md.compute_distances(traj, pairs)[0]
    mat = np.zeros((len(ca), len(ca)), dtype=int)
    for (i, j), dist in zip(pairs, d):
        if dist < thr:
            mat[i, j] = mat[j, i] = 1
    return mat


# --------------------------------------------------
# 5) Driver
# --------------------------------------------------


def analyze(pdb_path):
    traj, struct = load_structures(pdb_path)

    sasa, sasa_n = compute_sasa(traj, struct)
    phi, psi, om, chi1 = compute_torsions(traj, struct)
    hb = compute_hbonds(traj, struct)
    sb = compute_salt_bridges(traj, struct)
    ss = compute_ss_bridges(traj, struct)
    pp = compute_pi_pi(traj, struct)
    vdw = compute_vdw_contacts(traj, struct)
    pd = compute_packing_density(traj, struct)
    depth, protr = compute_depth_protrusion(struct)
    hsu, hsd = compute_half_sphere_exposure(struct)
    cmap = compute_contact_map(traj)

    rows = []
    for i, res in enumerate(struct.get_residues()):
        aa1 = resname_to_one(res.get_resname())
        rows.append(
            {
                "chain": res.get_parent().id,
                "resSeq": res.id[1],
                "resName": aa1,
                "ASA": sasa[i],
                "ASA_norm": sasa_n[i],
                "phi": phi[i],
                "psi": psi[i],
                "omega": om[i],
                "chi1": chi1[i],
                "Hbond_count": hb[i],
                "salt_bridge_count": sb[i],
                "ss_bridge": ss[i],
                "pi_pi_count": pp[i],
                "vdw_partners": vdw[i],
                "packing_density": pd[i],
                "hydrophobicity": HYDRO.get(aa1, np.nan),
                "depth": depth[i],
                "protrusion": protr[i],
                "HSE_up": hsu[i],
                "HSE_down": hsd[i],
            }
        )

    df = pd.DataFrame(rows)
    df.to_csv("per_residue_descriptors.csv", index=False)
    np.save("contact_map.npy", cmap)
    print("→ Saved 'per_residue_descriptors.csv' and 'contact_map.npy'")


if __name__ == "__main__":
    if len(sys.argv) != 2:
        print("Usage: python analyze_protein.py <pdb_file>")
        sys.exit(1)
    analyze(sys.argv[1])

Package 'biopython' not found. Installing...
Usage: python analyze_protein.py <pdb_file>


SystemExit: 1

/usr/local/lib/python3.11/dist-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


In [23]:
# First time only: install requirements
!pip install numpy pandas mdtraj biopython

# Import the script (it must be in the same folder or in your PYTHONPATH)
import analyze_protein

# Call the analyze function directly
analyze_protein.analyze("")  # replace with your filename

IndexError: index 0 is out of bounds for axis 0 with size 0

In [24]:
!pwd
!ls -al

/content
total 32
drwxr-xr-x 1 root root  4096 Apr 22 12:34 .
drwxr-xr-x 1 root root  4096 Apr 22 12:21 ..
-rw-r--r-- 1 root root 10704 Apr 22 12:36 analyze_protein.py
drwxr-xr-x 4 root root  4096 Apr 17 13:36 .config
drwxr-xr-x 2 root root  4096 Apr 22 12:34 __pycache__
drwxr-xr-x 1 root root  4096 Apr 17 13:36 sample_data


In [22]:
%%bash
cat > analyze_protein.py << 'EOF'
#!/usr/bin/env python3
"""
Comprehensive per‑residue structural descriptor calculation from a PDB,
with automatic installation of missing dependencies, and Biopython ≥ 1.80 support.

Usage:
    python analyze_protein.py <path/to/structure.pdb>
"""

import sys
import subprocess

#--------------------------------------------------
# 1) Auto‑install missing packages
#--------------------------------------------------
def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("numpy", "pandas", "mdtraj", "biopython"):
    try:
        __import__(pkg)
    except ImportError:
        print(f"Package '{pkg}' not found. Installing...")
        install(pkg)

# Now safe to import everything
import mdtraj as md
import numpy as np
import pandas as pd
from Bio.PDB import PDBParser, HSExposureCA
from Bio.Data.PDBData import protein_letters_3to1

#--------------------------------------------------
# 2) Constants
#--------------------------------------------------
MAX_ASA = {
    'A': 121, 'R': 265, 'N': 187, 'D': 187, 'C': 148, 'Q': 214, 'E': 223,
    'G': 97,  'H': 216, 'I': 195, 'L': 191, 'K': 230, 'M': 203, 'F': 228,
    'P': 154, 'S': 143, 'T': 163, 'W': 264, 'Y': 255, 'V': 165
}

HYDRO = {
    'A':  1.8,  'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
    'L': 3.8,  'K': -3.9, 'M': 1.9,  'F': 2.8,  'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

VDW_RADII = {'H': 1.2, 'C': 1.7, 'N': 1.55, 'O': 1.52, 'S': 1.8}

#--------------------------------------------------
# 3) Helper to map 3‑letter → 1‑letter
#--------------------------------------------------
def resname_to_one(resname):
    """Convert a three‑letter residue name to one‑letter code, default 'X'."""
    return protein_letters_3to1.get(resname.upper(), 'X')

#--------------------------------------------------
# 4) Core functions
#--------------------------------------------------
def load_structures(pdb_path):
    traj = md.load(pdb_path)
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('X', pdb_path)
    return traj, structure

def compute_sasa(traj, structure):
    sasa_nm2 = md.shrake_rupley(traj, probe_radius=0.14, mode='residue')
    sasa = sasa_nm2[0] * 100.0  # nm² → Å²
    norm = []
    for res in structure.get_residues():
        aa1 = resname_to_one(res.get_resname())
        max_ = MAX_ASA.get(aa1, np.nan)
        norm.append(sasa[res.id[1]-1] / max_ if max_ else np.nan)
    return sasa, np.array(norm)

def compute_torsions(traj, structure):
    phi_idx, phi_vals = md.compute_phi(traj)
    psi_idx, psi_vals = md.compute_psi(traj)
    om_idx, om_vals   = md.compute_omega(traj)
    chi1_idx, chi1_vals = md.compute_chi1(traj)

    phi   = dict(zip(phi_idx[:,1], phi_vals[0]))
    psi   = dict(zip(psi_idx[:,1], psi_vals[0]))
    omega = dict(zip(om_idx[:,1],   om_vals[0]))
    chi1  = dict(zip(chi1_idx[:,0], chi1_vals[0]))

    phis = []; psis = []; omegas = []; chis = []
    for res in structure.get_residues():
        i = res.id[1] - 1
        phis.append(phi.get(i, np.nan))
        psis.append(psi.get(i, np.nan))
        omegas.append(omega.get(i, np.nan))
        chis.append(chi1.get(i, np.nan))
    return (np.array(phis), np.array(psis),
            np.array(omegas), np.array(chis))

def compute_hbonds(traj, structure):
    """
    Count number of H‑bonds per residue (donor + acceptor).
    Works even if no H‑bonds are found.
    """
    # MDTraj baker_hubbard may return either:
    #   - an array of shape (n_bonds, 5)
    #   - or a tuple (array, frequencies)
    raw = md.baker_hubbard(traj, periodic=False)
    # unpack
    if isinstance(raw, tuple):
        hbonds = raw[0]
    else:
        hbonds = raw

    # Prepare a counter initialized to zero for every residue
    counts = {res.id[1]: 0 for res in structure.get_residues()}

    # If hbonds is empty, this loop simply does nothing
    for bond in hbonds:
        donor_idx   = int(bond[0])
        acceptor_idx= int(bond[2])
        dres = traj.topology.atom(donor_idx).residue.id
        ares = traj.topology.atom(acceptor_idx).residue.id
        counts[dres] += 1
        counts[ares] += 1

    # Build and return a numpy array in structure order
    return np.array([counts[r.id[1]] for r in structure.get_residues()])

def compute_salt_bridges(traj, structure, cutoff=0.4):
    neg = traj.topology.select("resname ASP GLU and name OD1 OD2 OE1 OE2")
    pos = traj.topology.select("resname LYS ARG HIS and name NZ NH1 NH2 NE")
    d = md.compute_distances(traj, np.array([[i,j] for i in neg for j in pos]))[0]
    pairs = np.where(d < cutoff)
    counts = {r.id[1]:0 for r in structure.get_residues()}
    for ni, pi in zip(*pairs):
        ri = traj.topology.atom(neg[ni]).residue.id
        rj = traj.topology.atom(pos[pi]).residue.id
        counts[ri] += 1; counts[rj] += 1
    return np.array([counts[r.id[1]] for r in structure.get_residues()])

def compute_ss_bridges(traj, structure, cutoff=0.22):
    sg = traj.topology.select("resname CYS and name SG")
    d = md.compute_distances(traj, np.array([[i,j] for i in sg for j in sg]))[0]
    pairs = np.where((d < cutoff) & (d > 0))
    bonded = set()
    for i, j in zip(*pairs):
        bonded.update([
            traj.topology.atom(sg[i]).residue.id,
            traj.topology.atom(sg[j]).residue.id
        ])
    return np.array([1 if r.id[1] in bonded else 0 for r in structure.get_residues()])

def compute_pi_pi(traj, structure, cutoff=0.7):
    arom = {
      'PHE':('CG','CD1','CD2','CE1','CE2','CZ'),
      'TYR':('CG','CD1','CD2','CE1','CE2','CZ'),
      'TRP':('CG','CD1','CD2','NE1','CE2','CE3','CZ2','CZ3','CH2')
    }
    cents = {}
    for chain in structure:
        for res in chain:
            name = res.get_resname()
            if name in arom:
                coords = np.array([a.get_vector() for a in res
                                   if a.get_name() in arom[name]])
                cents[res.id[1]] = coords.mean(axis=0)
    counts = {r.id[1]:0 for r in structure.get_residues()}
    keys = list(cents)
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            if np.linalg.norm(cents[keys[i]]-cents[keys[j]]) < cutoff*10:
                counts[keys[i]] += 1; counts[keys[j]] += 1
    return np.array([counts[r.id[1]] for r in structure.get_residues()])

def compute_vdw_contacts(traj, structure, extra=0.05):
    coords = traj.xyz[0] * 10
    elems  = [a.element.symbol for a in traj.topology.atoms]
    radii  = np.array([VDW_RADII.get(e,1.5) for e in elems])
    pairs = np.array([[i,j] for i in range(len(coords)) for j in range(i+1, len(coords))])
    dists = np.linalg.norm(coords[pairs[:,0]]-coords[pairs[:,1]], axis=1)
    thresh = radii[pairs[:,0]] + radii[pairs[:,1]] + extra
    hits   = pairs[dists < thresh]
    resmap = {a.index:a.residue.id[1] for a in traj.topology.atoms}
    conts  = {r.id[1]:set() for r in structure.get_residues()}
    for i,j in hits:
        ri, rj = resmap[i], resmap[j]
        if ri!=rj:
            conts[ri].add(rj); conts[rj].add(ri)
    return np.array([len(conts[r.id[1]]) for r in structure.get_residues()])

def compute_packing_density(traj, structure, r=1.0):
    ca = traj.topology.select("name CA")
    coords = traj.xyz[0]
    vol = 4/3 * np.pi * r**3 * 1e3
    pd = []
    for idx, res in enumerate(structure.get_residues()):
        d = np.linalg.norm((coords - coords[0][ca[idx]]), axis=2) * 10
        pd.append((np.sum(d<r*10)-1)/vol)
    return np.array(pd)

def compute_depth_protrusion(structure):
    atoms = np.array([a.get_vector() for a in structure.get_atoms()])
    com   = atoms.mean(axis=0)
    depths, protr = [], []
    for res in structure.get_residues():
        coords = np.array([a.get_vector() for a in res if a.element.symbol!='H'])
        d = np.linalg.norm(coords-com, axis=1)
        depths.append(d.mean()); protr.append(d.max()-d.mean())
    return np.array(depths), np.array(protr)

def compute_half_sphere_exposure(structure):
    model = structure[0]
    hs = HSExposureCA(model)
    up, down = [], []
    for res in model:
        u, d = hs[(res,'CA')]
        up.append(u); down.append(d)
    return np.array(up), np.array(down)

def compute_contact_map(traj, thr=0.8):
    ca = traj.topology.select("name CA")
    pairs = np.array([[i,j] for i in ca for j in ca if i<j])
    d = md.compute_distances(traj, pairs)[0]
    mat = np.zeros((len(ca), len(ca)), dtype=int)
    for (i,j), dist in zip(pairs, d):
        if dist < thr: mat[i,j] = mat[j,i] = 1
    return mat

#--------------------------------------------------
# 5) Driver
#--------------------------------------------------
def analyze(pdb_path):
    traj, struct = load_structures(pdb_path)

    sasa, sasa_n    = compute_sasa(traj, struct)
    phi, psi, om, chi1 = compute_torsions(traj, struct)
    hb   = compute_hbonds(traj, struct)
    sb   = compute_salt_bridges(traj, struct)
    ss   = compute_ss_bridges(traj, struct)
    pp   = compute_pi_pi(traj, struct)
    vdw  = compute_vdw_contacts(traj, struct)
    pd   = compute_packing_density(traj, struct)
    depth, protr    = compute_depth_protrusion(struct)
    hsu, hsd        = compute_half_sphere_exposure(struct)
    cmap            = compute_contact_map(traj)

    rows = []
    for i, res in enumerate(struct.get_residues()):
        aa1 = resname_to_one(res.get_resname())
        rows.append({
            'chain':             res.get_parent().id,
            'resSeq':            res.id[1],
            'resName':           aa1,
            'ASA':               sasa[i],
            'ASA_norm':          sasa_n[i],
            'phi':               phi[i],
            'psi':               psi[i],
            'omega':             om[i],
            'chi1':              chi1[i],
            'Hbond_count':       hb[i],
            'salt_bridge_count': sb[i],
            'ss_bridge':         ss[i],
            'pi_pi_count':       pp[i],
            'vdw_partners':      vdw[i],
            'packing_density':   pd[i],
            'hydrophobicity':    HYDRO.get(aa1, np.nan),
            'depth':             depth[i],
            'protrusion':        protr[i],
            'HSE_up':            hsu[i],
            'HSE_down':          hsd[i]
        })

    df = pd.DataFrame(rows)
    df.to_csv('per_residue_descriptors.csv', index=False)
    np.save('contact_map.npy', cmap)
    print("→ Saved 'per_residue_descriptors.csv' and 'contact_map.npy'")

if __name__ == '__main__':
    if len(sys.argv) != 2:
        print("Usage: python analyze_protein.py <pdb_file>")
        sys.exit(1)
    analyze(sys.argv[1])
EOF

In [ ]:
%%writefile analyze_protein.py
#!/usr/bin/env python3
"""
Comprehensive per-residue structural descriptor calculation from a PDB,
with automatic installation of missing dependencies, and Biopython ≥ 1.80 support.

Usage:
    python analyze_protein.py <path/to/structure.pdb>
"""

import sys
import subprocess

#--------------------------------------------------
# 1) Auto-install missing packages
#--------------------------------------------------
def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("numpy", "pandas", "mdtraj", "biopython"):
    try:
        __import__(pkg)
    except ImportError:
        print(f"Package '{pkg}' not found. Installing...")
        install(pkg)

import mdtraj as md
import numpy as np
import pandas as pd
from Bio.PDB import PDBParser, HSExposureCA
from Bio.Data.PDBData import protein_letters_3to1

#--------------------------------------------------
# 2) Constants
#--------------------------------------------------
MAX_ASA = {
    'A': 121, 'R': 265, 'N': 187, 'D': 187, 'C': 148, 'Q': 214, 'E': 223,
    'G': 97,  'H': 216, 'I': 195, 'L': 191, 'K': 230, 'M': 203, 'F': 228,
    'P': 154, 'S': 143, 'T': 163, 'W': 264, 'Y': 255, 'V': 165
}

HYDRO = {
    'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5, 'Q': -3.5, 'E': -3.5,
    'G': -0.4, 'H': -3.2, 'I': 4.5, 'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8,
    'P': -1.6, 'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

VDW_RADII = {'H': 1.2, 'C': 1.7, 'N': 1.55, 'O': 1.52, 'S': 1.8}

#--------------------------------------------------
# 3) Helper: 3-letter to 1-letter
#--------------------------------------------------
def resname_to_one(resname):
    return protein_letters_3to1.get(resname.upper(), 'X')

#--------------------------------------------------
# 4) Core functions
#--------------------------------------------------
def load_structures(pdb_path):
    traj = md.load(pdb_path)
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('X', pdb_path)
    return traj, structure

def compute_sasa(traj, structure):
    sasa_nm2 = md.shrake_rupley(traj, probe_radius=0.14, mode='residue')
    sasa = sasa_nm2[0] * 100.0
    residues = list(structure.get_residues())
    norm = []
    for i, res in enumerate(residues):
        aa1 = resname_to_one(res.get_resname())
        max_ = MAX_ASA.get(aa1, np.nan)
        norm.append(sasa[i] / max_ if max_ else np.nan)
    return sasa, np.array(norm)

def compute_torsions(traj, structure):
    phi_idx, phi_vals = md.compute_phi(traj)
    psi_idx, psi_vals = md.compute_psi(traj)
    om_idx, om_vals   = md.compute_omega(traj)
    chi1_idx, chi1_vals = md.compute_chi1(traj)

    phi   = dict(zip(phi_idx[:, 1], phi_vals[0]))
    psi   = dict(zip(psi_idx[:, 1], psi_vals[0]))
    omega = dict(zip(om_idx[:, 1], om_vals[0]))
    chi1  = dict(zip(chi1_idx[:, 0], chi1_vals[0]))

    results = []
    for i in range(len(list(structure.get_residues()))):
        results.append([
            phi.get(i, np.nan), psi.get(i, np.nan),
            omega.get(i, np.nan), chi1.get(i, np.nan)
        ])
    return map(np.array, zip(*results))

def compute_hbonds(traj, structure):
    hbonds = md.baker_hubbard(traj, periodic=False)
    counts = {res.index: 0 for res in traj.topology.residues}
    for i, j, k in hbonds:
        counts[traj.topology.atom(i).residue.index] += 1
        counts[traj.topology.atom(k).residue.index] += 1
    return np.array([counts[i] for i in range(len(counts))])

def compute_salt_bridges(traj, structure, cutoff=0.4):
    acidic = ['ASP', 'GLU']
    basic = ['ARG', 'LYS', 'HIS']
    pairs = []
    for i, a1 in enumerate(traj.topology.atoms):
        if a1.residue.name in acidic and a1.name in ['OD1', 'OD2', 'OE1', 'OE2']:
            for j, a2 in enumerate(traj.topology.atoms):
                if a2.residue.name in basic and a2.name in ['NZ', 'NH1', 'NH2', 'ND1', 'NE2']:
                    pairs.append([i, j])
    if not pairs:
        return np.zeros(len(list(structure.get_residues())), dtype=int)
    dists = md.compute_distances(traj, pairs)[0]
    from collections import defaultdict
    counts = defaultdict(int)
    for idx, (i, j) in enumerate(pairs):
        if dists[idx] < cutoff:
            counts[traj.topology.atom(i).residue.index] += 1
            counts[traj.topology.atom(j).residue.index] += 1
    result = np.zeros(len(list(structure.get_residues())), dtype=int)
    for k, v in counts.items():
        result[k] = v
    return result

def compute_ss_bridges(traj, structure, cutoff=0.22):
    sg = traj.topology.select("resname CYS and name SG")
    pairs = np.array([[i, j] for i in sg for j in sg if i < j])
    if not len(pairs): return np.zeros(len(list(structure.get_residues())), dtype=int)
    d = md.compute_distances(traj, pairs)[0]
    bonded = set()
    for (i, j), dist in zip(pairs, d):
        if 0 < dist < cutoff:
            bonded.update([traj.topology.atom(i).residue.index,
                           traj.topology.atom(j).residue.index])
    return np.array([1 if i in bonded else 0 for i in range(len(list(structure.get_residues())))])

def compute_pi_pi(traj, structure, cutoff=0.7):
    arom = {
        'PHE': ('CG','CD1','CD2','CE1','CE2','CZ'),
        'TYR': ('CG','CD1','CD2','CE1','CE2','CZ'),
        'TRP': ('CG','CD1','CD2','NE1','CE2','CE3','CZ2','CZ3','CH2')
    }
    cents = {}
    for chain in structure[0]:
        for res in chain:
            name = res.get_resname()
            if name in arom:
                coords = [a.get_coord() for a in res if a.get_name() in arom[name]]
                if coords:
                    cents[res.id[1]] = np.mean(coords, axis=0)
    keys = list(cents)
    counts = {k: 0 for k in keys}
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            if np.linalg.norm(cents[keys[i]] - cents[keys[j]]) < cutoff * 10:
                counts[keys[i]] += 1
                counts[keys[j]] += 1
    return np.array([counts.get(res.id[1], 0) for res in structure.get_residues()])

def compute_vdw_contacts(traj, structure, extra=0.05):
    coords = traj.xyz[0] * 10
    elems = [a.element.symbol for a in traj.topology.atoms]
    radii = np.array([VDW_RADII.get(e, 1.5) for e in elems])
    pairs = np.array([[i, j] for i in range(len(coords)) for j in range(i+1, len(coords))])
    dists = np.linalg.norm(coords[pairs[:,0]] - coords[pairs[:,1]], axis=1)
    thresh = radii[pairs[:,0]] + radii[pairs[:,1]] + extra
    hits = pairs[dists < thresh]
    resmap = {a.index: a.residue.index for a in traj.topology.atoms}
    residues = list(structure.get_residues())
    conts = {i: set() for i in range(len(residues))}
    for i, j in hits:
        ri, rj = resmap[i], resmap[j]
        if ri != rj:
            conts[ri].add(rj)
            conts[rj].add(ri)
    return np.array([len(conts[i]) for i in range(len(structure.get_residues()))])

def compute_packing_density(traj, structure, r=1.0):
    ca = traj.topology.select("name CA")
    coords = traj.xyz[0][ca] * 10
    vol = (4/3) * np.pi * (r ** 3)
    pd = []
    for i in range(len(coords)):
        d = np.linalg.norm(coords - coords[i], axis=1)
        pd.append((np.sum(d < r * 10) - 1) / vol)
    return np.array(pd)

def compute_depth_protrusion(structure):
    atoms = np.array([a.get_coord() for a in structure.get_atoms() if a.element != 'H'])
    com = atoms.mean(axis=0)
    depths, protr = [], []
    for res in structure.get_residues():
        coords = np.array([a.get_coord() for a in res if a.element != 'H'])
        d = np.linalg.norm(coords - com, axis=1)
        depths.append(d.mean()); protr.append(d.max() - d.mean())
    return np.array(depths), np.array(protr)

def compute_half_sphere_exposure(structure):
    model = structure[0]
    hs = HSExposureCA(model)
    up, down = [], []
    for res in model:
        try:
            u, d = hs[(res, 'CA')]
        except KeyError:
            u, d = np.nan, np.nan
        up.append(u)
        down.append(d)
    return np.array(up), np.array(down)

def compute_contact_map(traj, thr=0.8):
    ca = traj.topology.select("name CA")
    pairs = np.array([[i, j] for i in ca for j in ca if i < j])
    d = md.compute_distances(traj, pairs)[0]
    n = len(ca)
    mat = np.zeros((n, n), dtype=int)
    for (i, j), dist in zip(pairs, d):
        if dist < thr:
            i_idx = np.where(ca == i)[0][0]
            j_idx = np.where(ca == j)[0][0]
            mat[i_idx, j_idx] = mat[j_idx, i_idx] = 1
    return mat

#--------------------------------------------------
# 5) Driver
#--------------------------------------------------
def analyze(pdb_path):
    traj, struct = load_structures(pdb_path)
    sasa, sasa_n = compute_sasa(traj, struct)
    phi, psi, om, chi1 = compute_torsions(traj, struct)
    hb = compute_hbonds(traj, struct)
    sb = compute_salt_bridges(traj, struct)
    ss = compute_ss_bridges(traj, struct)
    pp = compute_pi_pi(traj, struct)
    vdw = compute_vdw_contacts(traj, struct)
    pd = compute_packing_density(traj, struct)
    depth, protr = compute_depth_protrusion(struct)
    hsu, hsd = compute_half_sphere_exposure(struct)
    cmap = compute_contact_map(traj)

    rows = []
    for i, res in enumerate(struct.get_residues()):
        aa1 = resname_to_one(res.get_resname())
        rows.append({
            'chain': res.get_parent().id,
            'resSeq': res.id[1],
            'resName': aa1,
            'ASA': sasa[i],
            'ASA_norm': sasa_n[i],
            'phi': phi[i],
            'psi': psi[i],
            'omega': om[i],
            'chi1': chi1[i],
            'Hbond_count': hb[i],
            'salt_bridge_count': sb[i],
            'ss_bridge': ss[i],
            'pi_pi_count': pp[i],
            'vdw_partners': vdw[i],
            'packing_density': pd[i],
            'hydrophobicity': HYDRO.get(aa1, np.nan),
            'depth': depth[i],
            'protrusion': protr[i],
            'HSE_up': hsu[i],
            'HSE_down': hsd[i]
        })

    df = pd.DataFrame(rows)
    df.to_csv('per_residue_descriptors.csv', index=False)
    np.save('contact_map.npy', cmap)
    print("→ Saved 'per_residue_descriptors.csv' and 'contact_map.npy'")

if __name__ == '__main__':
    if len(sys.argv) == 2:
        analyze(sys.argv[1])

In [ ]:
%%writefile analyze_protein.py
#!/usr/bin/env python3
"""
Comprehensive per-residue structural descriptor calculation from a PDB,
with automatic installation of missing dependencies, and Biopython ≥ 1.80 support.

Usage:
    python analyze_protein.py <path/to/structure.pdb>
"""

import sys
import subprocess

#--------------------------------------------------
# 1) Auto‑install missing packages
#--------------------------------------------------
def install(pkg):
    subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("numpy", "pandas", "mdtraj", "biopython"):
    try:
        __import__(pkg)
    except ImportError:
        print(f"Package '{pkg}' not found. Installing...")
        install(pkg)

import mdtraj as md
import numpy as np
import pandas as pd
from Bio.PDB import PDBParser, HSExposureCA
from Bio.Data.PDBData import protein_letters_3to1

#--------------------------------------------------
# 2) Constants
#--------------------------------------------------
MAX_ASA = {
    'A': 121, 'R': 265, 'N': 187, 'D': 187, 'C': 148, 'Q': 214, 'E': 223,
    'G': 97,  'H': 216, 'I': 195, 'L': 191, 'K': 230, 'M': 203, 'F': 228,
    'P': 154, 'S': 143, 'T': 163, 'W': 264, 'Y': 255, 'V': 165
}

HYDRO = {
    'A':  1.8,  'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
    'L': 3.8,  'K': -3.9, 'M': 1.9,  'F': 2.8,  'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

VDW_RADII = {'H': 1.2, 'C': 1.7, 'N': 1.55, 'O': 1.52, 'S': 1.8}

def resname_to_one(resname):
    return protein_letters_3to1.get(resname.upper(), 'X')

#--------------------------------------------------
# 3) Core Functions
#--------------------------------------------------
def load_structures(pdb_path):
    traj = md.load(pdb_path)
    parser = PDBParser(QUIET=True)
    structure = parser.get_structure('X', pdb_path)
    return traj, structure

def compute_sasa(traj, structure):
    sasa_nm2 = md.shrake_rupley(traj, probe_radius=0.14, mode='residue')
    sasa = sasa_nm2[0] * 100.0
    residues = list(structure.get_residues())
    norm = []
    for res in residues:
        aa1 = resname_to_one(res.get_resname())
        max_ = MAX_ASA.get(aa1, np.nan)
        norm.append(sasa[res.id[1]-1] / max_ if max_ else np.nan)
    return sasa, np.array(norm)

def compute_torsions(traj, structure):
    phi_idx, phi_vals = md.compute_phi(traj)
    psi_idx, psi_vals = md.compute_psi(traj)
    om_idx, om_vals   = md.compute_omega(traj)
    chi1_idx, chi1_vals = md.compute_chi1(traj)

    phi   = dict(zip(phi_idx[:,1], phi_vals[0]))
    psi   = dict(zip(psi_idx[:,1], psi_vals[0]))
    omega = dict(zip(om_idx[:,1],   om_vals[0]))
    chi1  = dict(zip(chi1_idx[:,0], chi1_vals[0]))

    results = list(structure.get_residues())
    phis = []; psis = []; omegas = []; chis = []
    for res in results:
        i = res.id[1] - 1
        phis.append(phi.get(i, np.nan))
        psis.append(psi.get(i, np.nan))
        omegas.append(omega.get(i, np.nan))
        chis.append(chi1.get(i, np.nan))
    return (np.array(phis), np.array(psis),
            np.array(omegas), np.array(chis))

def compute_hbonds(traj, structure):
    raw = md.baker_hubbard(traj, periodic=False)
    hbonds = raw[0] if isinstance(raw, tuple) else raw
    residues = list(structure.get_residues())
    counts = {res.id[1]: 0 for res in residues}
    for bond in hbonds:
        dres = traj.topology.atom(int(bond[0])).residue.index
        ares = traj.topology.atom(int(bond[2])).residue.index
        counts[dres] += 1
        counts[ares] += 1
    return np.array([counts.get(i, 0) for i in range(len(residues))])

def compute_salt_bridges(traj, structure, cutoff=0.4):
    from collections import defaultdict
    acidic = ['ASP', 'GLU']
    basic = ['ARG', 'LYS', 'HIS']

    pairs = []
    for i, atom_i in enumerate(traj.topology.atoms):
        if atom_i.residue.name in acidic and atom_i.name in ['OD1', 'OD2', 'OE1', 'OE2']:
            for j, atom_j in enumerate(traj.topology.atoms):
                if atom_j.residue.name in basic and atom_j.name in ['NZ', 'NH1', 'NH2', 'ND1', 'NE2']:
                    pairs.append([i, j])

    residues = list(structure.get_residues())
    if not pairs:
        return np.zeros(len(residues), dtype=int)

    distances = md.compute_distances(traj, pairs)[0]
    counts = defaultdict(int)
    for idx, (i, j) in enumerate(pairs):
        if distances[idx] < cutoff:
            res_i = traj.topology.atom(i).residue.index
            res_j = traj.topology.atom(j).residue.index
            counts[res_i] += 1
            counts[res_j] += 1

    result = np.zeros(len(residues), dtype=int)
    for i, c in counts.items():
        result[i] = c
    return result

def compute_ss_bridges(traj, structure, cutoff=0.22):
    sg = traj.topology.select("resname CYS and name SG")
    d = md.compute_distances(traj, np.array([[i,j] for i in sg for j in sg]))[0]
    pairs = np.where((d < cutoff) & (d > 0))
    bonded = set()
    for i, j in zip(*pairs):
        bonded.update([
            traj.topology.atom(sg[i]).residue.index,
            traj.topology.atom(sg[j]).residue.index
        ])
    residues = list(structure.get_residues())
    return np.array([1 if i in bonded else 0 for i in range(len(residues))])

def compute_pi_pi(traj, structure, cutoff=0.7):
    arom = {
        'PHE': ('CG','CD1','CD2','CE1','CE2','CZ'),
        'TYR': ('CG','CD1','CD2','CE1','CE2','CZ'),
        'TRP': ('CG','CD1','CD2','NE1','CE2','CE3','CZ2','CZ3','CH2')
    }
    cents = {}
    residues = list(structure.get_residues())
    for chain in structure[0]:
        for res in chain:
            name = res.get_resname()
            if name in arom:
                coords = np.array([a.get_vector() for a in res if a.get_name() in arom[name]])
                if len(coords):
                    cents[res.id[1]] = coords.mean(axis=0)
    counts = {r.id[1]: 0 for r in residues}
    keys = list(cents.keys())
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            if np.linalg.norm(cents[keys[i]] - cents[keys[j]]) < cutoff * 10:
                counts[keys[i]] += 1
                counts[keys[j]] += 1
    return np.array([counts[r.id[1]] for r in residues])

def compute_vdw_contacts(traj, structure, extra=0.05):
    coords = traj.xyz[0] * 10
    elems  = [a.element.symbol for a in traj.topology.atoms]
    radii  = np.array([VDW_RADII.get(e,1.5) for e in elems])
    pairs = np.array([[i,j] for i in range(len(coords)) for j in range(i+1, len(coords))])
    dists = np.linalg.norm(coords[pairs[:,0]]-coords[pairs[:,1]], axis=1)
    thresh = radii[pairs[:,0]] + radii[pairs[:,1]] + extra
    hits   = pairs[dists < thresh]
    residues = list(structure.get_residues())
    resmap = {a.index:a.residue.index for a in traj.topology.atoms}
    conts  = {i:set() for i in range(len(residues))}
    for i,j in hits:
        ri, rj = resmap[i], resmap[j]
        if ri != rj:
            conts[ri].add(rj); conts[rj].add(ri)
    return np.array([len(conts[i]) for i in range(len(residues))])

def compute_packing_density(traj, structure, r=1.0):
    ca_indices = traj.topology.select("name CA")
    coords = traj.xyz[0][ca_indices]  # only CA atoms
    vol = 4/3 * np.pi * r**3 * 1e3
    pdc = np.zeros(len(list(structure.get_residues())))  # full-length array with zeros
    ca_res_map = {i: atom.residue.index for i, atom in enumerate(traj.topology.atoms) if atom.name == "CA"}

    for idx, res_idx in enumerate(ca_res_map.values()):
        d = np.linalg.norm(coords - coords[idx], axis=1) * 10
        pdc[res_idx] = (np.sum(d < r * 10) - 1) / vol  # exclude self
    return pdc

def compute_depth_protrusion(structure):
    atoms = np.array([a.get_vector().get_array() for a in structure.get_atoms()])
    com   = atoms.mean(axis=0)
    depths, protr = [], []
    for res in structure.get_residues():
        coords = np.array([a.get_vector().get_array() for a in res if a.element != 'H'])
        d = np.linalg.norm(coords - com, axis=1)
        depths.append(d.mean())
        protr.append(d.max() - d.mean())
    return np.array(depths), np.array(protr)

def compute_half_sphere_exposure(structure):
    model = structure[0]
    hs = HSExposureCA(model)
    up, down = [], []
    for res in model.get_residues():
        try:
            u, d = hs[(res, 'CA')]
            up.append(u)
            down.append(d)
        except KeyError:
            up.append(np.nan)
            down.append(np.nan)
    return np.array(up), np.array(down)

def compute_contact_map(traj, thr=0.8):
    ca = traj.topology.select("name CA")
    index_map = {atom_idx: i for i, atom_idx in enumerate(ca)}
    pairs = np.array([[i, j] for i in ca for j in ca if i < j])
    dists = md.compute_distances(traj, pairs)[0]

    mat = np.zeros((len(ca), len(ca)), dtype=int)
    for (i_atom, j_atom), dist in zip(pairs, dists):
        i = index_map[i_atom]
        j = index_map[j_atom]
        if dist < thr:
            mat[i, j] = mat[j, i] = 1
    return mat

#--------------------------------------------------
# 4) Driver
#--------------------------------------------------
def analyze(pdb_path):
    traj, struct = load_structures(pdb_path)
    residues = list(struct.get_residues())

    sasa, sasa_n    = compute_sasa(traj, struct)
    phi, psi, om, chi1 = compute_torsions(traj, struct)
    hb   = compute_hbonds(traj, struct)
    sb   = compute_salt_bridges(traj, struct)
    ss   = compute_ss_bridges(traj, struct)
    pp   = compute_pi_pi(traj, struct)
    vdw  = compute_vdw_contacts(traj, struct)
    pdc   = compute_packing_density(traj, struct)
    depth, protr    = compute_depth_protrusion(struct)
    hsu, hsd        = compute_half_sphere_exposure(struct)
    cmap            = compute_contact_map(traj)

    rows = []
    for i, res in enumerate(residues):
        aa1 = resname_to_one(res.get_resname())
        rows.append({
            'chain':             res.get_parent().id,
            'resSeq':            res.id[1],
            'resName':           aa1,
            'ASA':               sasa[i],
            'ASA_norm':          sasa_n[i],
            'phi':               phi[i],
            'psi':               psi[i],
            'omega':             om[i],
            'chi1':              chi1[i],
            'Hbond_count':       hb[i],
            'salt_bridge_count': sb[i],
            'ss_bridge':         ss[i],
            'pi_pi_count':       pp[i],
            'vdw_partners':      vdw[i],
            'packing_density':   pdc[i],
            'hydrophobicity':    HYDRO.get(aa1, np.nan),
            'depth':             depth[i],
            'protrusion':        protr[i],
            'HSE_up':            hsu[i],
            'HSE_down':          hsd[i]
        })

    df = pd.DataFrame(rows)
    df.to_csv('per_residue_descriptors.csv', index=False)
    np.save('contact_map.npy', cmap)
    print("→ Saved 'per_residue_descriptors.csv' and 'contact_map.npy'")

if __name__ == '__main__':
    if len(sys.argv) == 2:
        analyze(sys.argv[1])

In [ ]:
%%writefile analyze_protein.py
#!/usr/bin/env python3
"""
Comprehensive per-residue and global structural descriptor analysis from a PDB file.
Requires: mdtraj, biopython, pandas, numpy, pdbfixer, openmm
"""

import os
import sys
import subprocess

def install(pkg): subprocess.check_call([sys.executable, "-m", "pip", "install", pkg])

for pkg in ("numpy", "pandas", "mdtraj", "biopython", "pdbfixer", "openmm"):
    try: __import__(pkg)
    except ImportError: install(pkg)

import numpy as np
import pandas as pd
import mdtraj as md
from Bio.PDB import PDBParser, HSExposureCA
from Bio.Data.PDBData import protein_letters_3to1
from pdbfixer import PDBFixer
from openmm.app import PDBFile

# Constants
HYDRO = {
    'A': 1.8, 'R': -4.5, 'N': -3.5, 'D': -3.5, 'C': 2.5,
    'Q': -3.5, 'E': -3.5, 'G': -0.4, 'H': -3.2, 'I': 4.5,
    'L': 3.8, 'K': -3.9, 'M': 1.9, 'F': 2.8, 'P': -1.6,
    'S': -0.8, 'T': -0.7, 'W': -0.9, 'Y': -1.3, 'V': 4.2
}

MAX_ASA = {
    'A': 121, 'R': 265, 'N': 187, 'D': 187, 'C': 148, 'Q': 214, 'E': 223,
    'G': 97,  'H': 216, 'I': 195, 'L': 191, 'K': 230, 'M': 203, 'F': 228,
    'P': 154, 'S': 143, 'T': 163, 'W': 264, 'Y': 255, 'V': 165
}

VDW_RADII = {'H': 1.2, 'C': 1.7, 'N': 1.55, 'O': 1.52, 'S': 1.8}

def resname_to_one(resname): return protein_letters_3to1.get(resname.upper(), 'X')

def fix_and_load_structure(pdb_path):
    fixer = PDBFixer(filename=pdb_path)
    fixer.findMissingResidues()
    fixer.findMissingAtoms()
    fixer.addMissingAtoms()
    fixer.addMissingHydrogens(pH=7.4)
    fixed_path = "fixed_structure.pdb"
    with open(fixed_path, "w") as f:
        PDBFile.writeFile(fixer.topology, fixer.positions, f)
    traj = md.load(fixed_path)
    structure = PDBParser(QUIET=True).get_structure("X", fixed_path)
    return traj, structure

def compute_sasa(traj, structure):
    sasa_nm2 = md.shrake_rupley(traj, probe_radius=0.14, mode='residue')
    sasa = sasa_nm2[0] * 100.0
    residues = list(structure.get_residues())
    norm = [sasa[i] / MAX_ASA.get(resname_to_one(r.get_resname()), np.nan) for i, r in enumerate(residues)]
    return sasa, np.array(norm)

def compute_torsions(traj, structure):
    torsion_types = [md.compute_phi, md.compute_psi, md.compute_omega, md.compute_chi1]
    torsion_data = {}
    for func in torsion_types:
        idx, vals = func(traj)
        for i, val in zip(idx[:,1], vals[0]):
            torsion_data.setdefault(i, []).append(val)
    residues = list(structure.get_residues())
    results = [[*torsion_data.get(i, [np.nan]*4)] for i in range(len(residues))]
    return np.array(results).T

def compute_hbonds(traj):
    hb = md.baker_hubbard(traj, periodic=False)
    counts = {}
    for i, j, k in hb:
        for atom in (i, k):
            res = traj.topology.atom(atom).residue.index
            counts[res] = counts.get(res, 0) + 1
    return counts

def compute_salt_bridges(traj):
    acidic = ["ASP", "GLU"]
    basic = ["ARG", "LYS", "HIS"]
    atoms = traj.topology.atoms
    pairs = [(i, j) for i in range(len(atoms)) for j in range(i+1, len(atoms))
             if atoms[i].residue.name in acidic and atoms[j].residue.name in basic]
    if not pairs: return {}
    dists = md.compute_distances(traj, pairs)[0]
    sb_counts = {}
    for (i, j), dist in zip(pairs, dists):
        if dist < 0.4:
            for idx in (i, j):
                res = traj.topology.atom(idx).residue.index
                sb_counts[res] = sb_counts.get(res, 0) + 1
    return sb_counts

def compute_disulfides(traj):
    sg = traj.topology.select("resname CYS and name SG")
    if len(sg) < 2: return {}
    pairs = [(i, j) for i in sg for j in sg if i < j]
    dists = md.compute_distances(traj, pairs)[0]
    bonded = {traj.topology.atom(sg[i]).residue.index for idx, (i, j) in enumerate(pairs) if 0 < dists[idx] < 0.22}
    return {idx: 1 for idx in bonded}

def compute_pi_pi(traj, structure):
    aromatics = {"PHE", "TYR", "TRP"}
    centers = {}
    for chain in structure[0]:
        for res in chain:
            if res.get_resname() in aromatics:
                coords = np.array([a.get_vector().get_array() for a in res if a.element != 'H'])
                centers[res.id[1]] = coords.mean(axis=0) if len(coords) > 0 else None
    counts = {k: 0 for k in centers}
    keys = list(centers)
    for i in range(len(keys)):
        for j in range(i+1, len(keys)):
            if np.linalg.norm(centers[keys[i]] - centers[keys[j]]) < 7.0:
                counts[keys[i]] += 1
                counts[keys[j]] += 1
    return counts

def compute_vdw_contacts(traj):
    coords = traj.xyz[0] * 10
    elems = [a.element.symbol for a in traj.topology.atoms]
    radii = np.array([VDW_RADII.get(e, 1.5) for e in elems])
    resmap = {a.index: a.residue.index for a in traj.topology.atoms}
    contacts = {i: set() for i in set(resmap.values())}
    for i in range(len(coords)):
        for j in range(i+1, len(coords)):
            if resmap[i] == resmap[j]: continue
            if np.linalg.norm(coords[i] - coords[j]) < (radii[i] + radii[j] + 0.05):
                contacts[resmap[i]].add(resmap[j])
                contacts[resmap[j]].add(resmap[i])
    return {k: len(v) for k, v in contacts.items()}

def compute_packing_density(traj):
    ca = traj.topology.select("name CA")
    coords = traj.xyz[0][ca] * 10
    vol = 4 / 3 * np.pi * 1.0**3 * 1e3
    densities = np.zeros(len(traj.topology.residues))
    for i, res in enumerate(ca):
        d = np.linalg.norm(coords - coords[i], axis=1)
        densities[i] = (np.sum(d < 10) - 1) / vol
    return densities

def compute_depth_protrusion(structure):
    atoms = np.array([a.get_vector().get_array() for a in structure.get_atoms()])
    com = atoms.mean(axis=0)
    depths, protr = [], []
    for res in structure.get_residues():
        coords = np.array([a.get_vector().get_array() for a in res if a.element != 'H'])
        d = np.linalg.norm(coords - com, axis=1)
        depths.append(d.mean())
        protr.append(d.max() - d.mean())
    return np.array(depths), np.array(protr)

def compute_half_sphere_exposure(structure):
    hse = HSExposureCA(structure[0])
    up, down = [], []
    for res in structure.get_residues():
        try:
            u, d = hse[(res, 'CA')]
            up.append(u)
            down.append(d)
        except:
            up.append(np.nan); down.append(np.nan)
    return np.array(up), np.array(down)

def compute_secondary_structure(traj):
    ss = traj.secondary_structure()[0]
    total = len(ss)
    content = {k: round((ss == k).sum() / total, 3) for k in 'HEC'}
    return content

def compute_contact_order(traj):
    ca = traj.topology.select("name CA")
    if len(ca) < 2: return 0
    pairs = np.array([[i, j] for i in ca for j in ca if abs(i - j) >= 3])
    dists = md.compute_distances(traj, pairs)[0]
    contact_pairs = pairs[dists < 0.8]
    sum_seq_sep = np.sum(np.abs(contact_pairs[:, 0] - contact_pairs[:, 1]))
    return round(sum_seq_sep / (len(contact_pairs) * len(ca)), 3) if len(contact_pairs) else 0

def analyze(pdb_path):
    traj, struct = fix_and_load_structure(pdb_path)
    residues = list(struct.get_residues())
    sasa, sasa_norm = compute_sasa(traj, struct)
    phi, psi, omega, chi1 = compute_torsions(traj, struct)
    hb = compute_hbonds(traj)
    sb = compute_salt_bridges(traj)
    ss = compute_disulfides(traj)
    pi = compute_pi_pi(traj, struct)
    vdw = compute_vdw_contacts(traj)
    pdens = compute_packing_density(traj)
    depth, protr = compute_depth_protrusion(struct)
    hsu, hsd = compute_half_sphere_exposure(struct)

    data = []
    for i, res in enumerate(residues):
        aa1 = resname_to_one(res.get_resname())
        data.append({
            "chain": res.get_parent().id,
            "resSeq": res.id[1],
            "resName": aa1,
            "ASA": sasa[i],
            "ASA_norm": sasa_norm[i],
            "phi": phi[i],
            "psi": psi[i],
            "omega": omega[i],
            "chi1": chi1[i],
            "Hbond_count": hb.get(i, 0),
            "salt_bridge_count": sb.get(i, 0),
            "ss_bridge": ss.get(i, 0),
            "pi_pi_count": pi.get(i, 0),
            "vdw_partners": vdw.get(i, 0),
            "packing_density": pdens[i],
            "depth": depth[i],
            "protrusion": protr[i],
            "hydrophobicity": HYDRO.get(aa1, np.nan),
            "HSE_up": hsu[i],
            "HSE_down": hsd[i]
        })

    df = pd.DataFrame(data)
    df.to_csv("per_residue_descriptors.csv", index=False)

    # Global descriptors
    ss_content = compute_secondary_structure(traj)
    contact_order = compute_contact_order(traj)
    global_desc = {
        "Total_SASA": round(sasa.sum(), 2),
        "Alpha_helix_content": ss_content.get('H', 0.0),
        "Beta_strand_content": ss_content.get('E', 0.0),
        "Coil_content": ss_content.get('C', 0.0),
        "Contact_order": contact_order,
        "Total_Hbonds": sum(hb.values()),
        "Total_salt_bridges": sum(sb.values()),
        "Total_ss_bridges": sum(ss.values()) // 2,
        "Total_pi_pi": sum(pi.values()) // 2
    }
    pd.Series(global_desc).to_csv("global_descriptors.csv")
    print("→ Saved per_residue_descriptors.csv and global_descriptors.csv")

if __name__ == "__main__":
    if len(sys.argv) == 2:
        analyze(sys.argv[1])
    else:
        print("Usage: python analyze_protein.py <structure.pdb>")